In [3]:
import os
from dotenv import load_dotenv

# Load biến môi trường từ file .env
load_dotenv()

# Gán giá trị API key vào biến môi trường
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [ ]:
%pip install -q youtube-transcript-api langchain-community langchain-google-genai faiss-cpu tiktoken python-dotenv

In [5]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

In [6]:
video_id = "L7Jp38FvmnM" # Bạn thay ID video của bạn vào đây

try:
    # Lấy transcript với ngôn ngữ ưu tiên là tiếng Việt ("vi")
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=["vi"])
    
    # Nối các đoạn text rời rạc thành một đoạn văn bản dài
    transcript = " ".join([chunk["text"] for chunk in transcript_list])
    
    print(transcript)

except TranscriptsDisabled:
    print("No captions available for this video.")
except Exception as e:
    print(f"Lỗi khác: {e}")

# Dòng này dùng để xem cấu trúc dữ liệu gốc (nếu chạy thành công)
if 'transcript_list' in locals():
    print(transcript_list)

Lỗi khác: type object 'YouTubeTranscriptApi' has no attribute 'get_transcript'


In [ ]:
# Tạo bộ chia văn bản
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Thực hiện chia nhỏ đoạn transcript (đã lấy ở bước trước)
chunks = splitter.create_documents([transcript])

# Kiểm tra xem có bao nhiêu đoạn (tùy chọn)
print(f"Số lượng đoạn văn nhỏ: {len(chunks)}")

In [ ]:
# Nhét vào vector DB
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001") #<-- embedding LLM
vector_store = FAISS.from_documents(chunks, embeddings)

print("Đã tạo xong Vector Store!")

In [ ]:
# Tạo công cụ tìm kiếm (Retriever), lấy 4 kết quả giống nhất (k=4)
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

results = retriever.invoke('cách đi thi PMP')

print(results)

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)


prompt = PromptTemplate(
    template="""
    You are a helpful assistant.
    Answer ONLY from the provided transcript context.
    If the context is insufficient, just say you don't know.

    {context}
    Question: {question}
    """,
    input_variables = ['context', 'question']
)

print("Đã cấu hình xong LLM và Prompt!")

In [ ]:
question = "Chi phí học tại PMA là bao nhiêu?"

# 2. Tìm kiếm các đoạn văn bản liên quan trong Video (Retrieval)
retrieved_docs = retriever.invoke(question)

# 3. Ghép nội dung các đoạn tìm được thành 1 văn bản dài (Context)
context_text = "\n\n".join([doc.page_content for doc in retrieved_docs])

# 4. Tạo prompt hoàn chỉnh (kết hợp Context + Câu hỏi)
final_prompt = prompt.invoke({"context": context_text, "question": question})

# 5. Gửi cho AI trả lời
answer = llm.invoke(final_prompt)

# 6. In kết quả ra màn hình
print("--- Câu trả lời của AI ---")
print(answer.content)